# Manipulate wind and solar farm data into something usable

In [1]:
import pandas as pd

In [3]:
path = "/g/data/w42/dr6273/work/data/Open_NEM/"

# Issues:

- Some farms are missing coordinates. Try and find them using the farm name and Google maps.
- Some farms have multiple energy types, but the Generator Capacity lists the total. Split these into separate entries, and assign correct capacities using OpenNem website.
- It seems like the facilities on the OpenNEM website change regularly. For the download on 07/06/2023, there was a wind farm called 'Crudine Ridge'. This wasn't present the next day...

# NSW

In [169]:
nsw = pd.read_csv(
    path+"facilities_solar_wind_NSW.csv"
)

### Coordinates

In [170]:
nsw.loc[
    (nsw['Latitude'].isnull()) |
    (nsw['Longitude'].isnull())
]

,Facility Name,Status,Region,Technology,Generator Capacity (MW),Latitude,Longitude
6,Bomen,operating,New South Wales,Solar (Utility),121.0,NaN,NaN
19,Goonumbla,operating,New South Wales,Solar (Utility),85.0,NaN,NaN
31,Metz N,operating,New South Wales,Solar (Utility),135.0,NaN,NaN
50,Wellington,operating,New South Wales,Solar (Utility),216.0,NaN,NaN
51,Wellington North,operating,New South Wales,Solar (Utility),436.8,NaN,NaN
53,White Rock,operating,New South Wales,"Solar (Utility),Wind",197.0,NaN,NaN


In [171]:
pd.options.mode.chained_assignment = None  # default='warn'

In [172]:
nsw_edited = nsw.copy()

Wellington

In [173]:
nsw_edited["Latitude"][nsw_edited["Facility Name"] == "Wellington"] = -32.5162961812663
nsw_edited["Longitude"][nsw_edited["Facility Name"] == "Wellington"] = 148.9612093835598

Wellington North (adjacent to Wellington)

In [174]:
nsw_edited["Latitude"][nsw_edited["Facility Name"] == "Wellington North"] = -32.5162961812663
nsw_edited["Longitude"][nsw_edited["Facility Name"] == "Wellington North"] = 148.9612093835598

White Rock

In [175]:
nsw_edited["Latitude"][nsw_edited["Facility Name"] == "White Rock"] = -29.76264055097494
nsw_edited["Longitude"][nsw_edited["Facility Name"] == "White Rock"] = 151.5521448079424

Metz N (approximated from http://metzsolarfarm.com/location/ and Google Maps)

In [176]:
nsw_edited["Latitude"][nsw_edited["Facility Name"] == "Metz N"] = -30.527104988268896
nsw_edited["Longitude"][nsw_edited["Facility Name"] == "Metz N"] = 151.87411854905747

Bomen

In [177]:
nsw_edited["Latitude"][nsw_edited["Facility Name"] == "Bomen"] = -35.048297036400946
nsw_edited["Longitude"][nsw_edited["Facility Name"] == "Bomen"] = 147.4442568460288

Goonumbla (http://goonumblasolarfarm.com/about-us/ and https://www.google.com/maps/place/Parkes+Solar+Farm/@-33.1115994,148.0872867,4400m/data=!3m1!1e3!4m15!1m8!3m7!1s0x6b1ab4449ce55405:0x40609b490437280!2sGoonumbla+NSW+2870!3b1!8m2!3d-32.9178827!4d148.0521359!16s%2Fm%2F012l1gyt!3m5!1s0x6b1ab947765bed85:0xceccccac6dd3e0a6!8m2!3d-33.1129309!4d148.0763972!16s%2Fg%2F11g9dn4xby?entry=ttu)

In [178]:
nsw_edited["Latitude"][nsw_edited["Facility Name"] == "Goonumbla"] = -33.11247426196654
nsw_edited["Longitude"][nsw_edited["Facility Name"] == "Goonumbla"] = 148.0764769027703

### Wind and solar

- look at all rows that dont solely have either Wind or Solar (Utility)

In [179]:
nsw.loc[
    (nsw["Technology"] != "Solar (Utility)") &
    (nsw["Technology"] != "Wind")
]

,Facility Name,Status,Region,Technology,Generator Capacity (MW),Latitude,Longitude
21,Gullen Range,operating,New South Wales,"Solar (Utility),Wind",175.0,-34.624512,149.458549
53,White Rock,operating,New South Wales,"Solar (Utility),Wind",197.0,NaN,NaN


Duplicate rows

In [180]:
nsw_edited = nsw_edited.append(
    [
        nsw_edited.loc[
            (nsw_edited["Technology"] != "Solar (Utility)") &
            (nsw_edited["Technology"] != "Wind")
        ]
    ],
    ignore_index=True
)

/jobfs/120005759.gadi-pbs/ipykernel_1286946/3926266531.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nsw_edited = nsw_edited.append(


In [181]:
nsw_edited.loc[nsw_edited["Technology"] == "Solar (Utility),Wind"]

,Facility Name,Status,Region,Technology,Generator Capacity (MW),Latitude,Longitude
21,Gullen Range,operating,New South Wales,"Solar (Utility),Wind",175.0,-34.624512,149.458549
53,White Rock,operating,New South Wales,"Solar (Utility),Wind",197.0,-29.762641,151.552145
56,Gullen Range,operating,New South Wales,"Solar (Utility),Wind",175.0,-34.624512,149.458549
57,White Rock,operating,New South Wales,"Solar (Utility),Wind",197.0,-29.762641,151.552145


Edit to separate out wind and solar
- White Rock: https://opennem.org.au/facilities/au/?selected=WHIROCWIND&tech=solar_utility,wind&status=operating
- Gullen Range: https://opennem.org.au/facilities/nsw1/?selected=GULLRGWF&tech=solar_utility,wind&status=operating

In [182]:
nsw_edited["Technology"].iloc[21] = "Solar (Utility)"
nsw_edited["Generator Capacity (MW)"].iloc[21] = 10

In [183]:
nsw_edited["Technology"].iloc[53] = "Solar (Utility)"
nsw_edited["Generator Capacity (MW)"].iloc[53] = 22

In [184]:
nsw_edited["Technology"].iloc[56] = "Wind"
nsw_edited["Generator Capacity (MW)"].iloc[56] = 165

In [185]:
nsw_edited["Technology"].iloc[57] = "Wind"
nsw_edited["Generator Capacity (MW)"].iloc[57] = 175

### Write

In [187]:
nsw_edited.to_csv(
    path+"edited/facilities_solar_wind_NSW_edited.csv",
    index=False
)

# QLD

In [188]:
qld = pd.read_csv(
    path+"facilities_solar_wind_QLD.csv"
)

### Coordinates

In [189]:
qld.loc[
    (qld['Latitude'].isnull()) |
    (qld['Longitude'].isnull())
]

,Facility Name,Status,Region,Technology,Generator Capacity (MW),Latitude,Longitude


In [190]:
qld_edited = qld.copy()

### Wind and solar

In [191]:
qld.loc[
    (qld["Technology"] != "Solar (Utility)") &
    (qld["Technology"] != "Wind")
]

,Facility Name,Status,Region,Technology,Generator Capacity (MW),Latitude,Longitude
20,Kennedy Energy Park,operating,Queensland,"Battery (Discharging),Solar (Utility),Wind",60.2,-20.872792,144.413034


Duplicate rows

In [192]:
qld_edited = qld_edited.append(
    [
        qld_edited.loc[
            (qld_edited["Technology"] != "Solar (Utility)") &
            (qld_edited["Technology"] != "Wind")
        ]
    ],
    ignore_index=True
)

/jobfs/120005759.gadi-pbs/ipykernel_1286946/3591010649.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  qld_edited = qld_edited.append(


In [193]:
qld_edited.loc[
    (qld_edited["Technology"] != "Solar (Utility)") &
    (qld_edited["Technology"] != "Wind")
]

,Facility Name,Status,Region,Technology,Generator Capacity (MW),Latitude,Longitude
20,Kennedy Energy Park,operating,Queensland,"Battery (Discharging),Solar (Utility),Wind",60.2,-20.872792,144.413034
43,Kennedy Energy Park,operating,Queensland,"Battery (Discharging),Solar (Utility),Wind",60.2,-20.872792,144.413034


In [194]:
qld_edited["Technology"].iloc[20] = "Solar (Utility)"
qld_edited["Generator Capacity (MW)"].iloc[20] = 15

In [195]:
qld_edited["Technology"].iloc[43] = "Wind"
qld_edited["Generator Capacity (MW)"].iloc[43] = 43

### Write

In [196]:
qld_edited.to_csv(
    path+"edited/facilities_solar_wind_QLD_edited.csv",
    index=False
)

# SA

In [204]:
sa = pd.read_csv(
    path+"facilities_solar_wind_SA.csv"
)

### Coordinates

In [205]:
sa.loc[
    (sa['Latitude'].isnull()) |
    (sa['Longitude'].isnull())
]

,Facility Name,Status,Region,Technology,Generator Capacity (MW),Latitude,Longitude
19,Morganwhyalla Pipeline,operating,South Australia,Solar (Utility),0.0,NaN,NaN


In [206]:
sa_edited = sa.copy()

Morganwhyalla Pipeline

- Has zero capacity so just remove

In [207]:
sa_edited = sa_edited.drop(labels=19, axis=0)

### Wind and solar

In [208]:
sa.loc[
    (sa["Technology"] != "Solar (Utility)") &
    (sa["Technology"] != "Wind")
]

,Facility Name,Status,Region,Technology,Generator Capacity (MW),Latitude,Longitude
0,Adelaide Desalination,operating,South Australia,"Battery (Discharging),Solar (Utility)",9.140,-35.096948,138.484061
1,Bolivar,committed,South Australia,"Battery (Discharging),Bioenergy (Biogas),Solar...",21.330,-34.495482,138.387641
5,Christies Beach,committed,South Australia,"Battery (Discharging),Solar (Utility)",4.190,-35.125958,138.471350
10,Happy Valley,committed,South Australia,"Battery (Discharging),Solar (Utility)",16.518,-35.063118,138.564760
23,Port Augusta Energy Park,operating,South Australia,"Solar (Utility),Wind",309.000,-32.595609,137.885629
29,Tailem Bend 2,operating,South Australia,"Battery (Discharging),Solar (Utility)",155.000,-35.280000,139.490000


Duplicate rows

In [209]:
# sa_edited = sa_edited.append(
#     [
#         sa_edited.loc[
#             (sa_edited["Technology"] != "Solar (Utility)") &
#             (sa_edited["Technology"] != "Wind")
#         ]
#     ],
#     ignore_index=True
# )

In [210]:
sa_edited = sa_edited.append(
    [
        sa_edited.loc[
            sa_edited["Technology"] == "Solar (Utility),Wind"
        ]
    ],
    ignore_index=True
)

/jobfs/120005759.gadi-pbs/ipykernel_1286946/1882836538.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sa_edited = sa_edited.append(


In [211]:
sa_edited.loc[
    (sa_edited["Technology"] != "Solar (Utility)") &
    (sa_edited["Technology"] != "Wind")
]

,Facility Name,Status,Region,Technology,Generator Capacity (MW),Latitude,Longitude
0,Adelaide Desalination,operating,South Australia,"Battery (Discharging),Solar (Utility)",9.140,-35.096948,138.484061
1,Bolivar,committed,South Australia,"Battery (Discharging),Bioenergy (Biogas),Solar...",21.330,-34.495482,138.387641
5,Christies Beach,committed,South Australia,"Battery (Discharging),Solar (Utility)",4.190,-35.125958,138.471350
10,Happy Valley,committed,South Australia,"Battery (Discharging),Solar (Utility)",16.518,-35.063118,138.564760
22,Port Augusta Energy Park,operating,South Australia,"Solar (Utility),Wind",309.000,-32.595609,137.885629
28,Tailem Bend 2,operating,South Australia,"Battery (Discharging),Solar (Utility)",155.000,-35.280000,139.490000
33,Port Augusta Energy Park,operating,South Australia,"Solar (Utility),Wind",309.000,-32.595609,137.885629


In [212]:
sa_edited["Technology"].iloc[0] = "Solar (Utility)"
sa_edited["Generator Capacity (MW)"].iloc[0] = 2.9

In [213]:
sa_edited["Technology"].iloc[1] = "Solar (Utility)"
sa_edited["Generator Capacity (MW)"].iloc[1] = 8.3

In [214]:
sa_edited["Technology"].iloc[5] = "Solar (Utility)"
sa_edited["Generator Capacity (MW)"].iloc[5] = 2.1

In [215]:
sa_edited["Technology"].iloc[10] = "Solar (Utility)"
sa_edited["Generator Capacity (MW)"].iloc[10] = 11

In [216]:
sa_edited["Technology"].iloc[22] = "Solar (Utility)"
sa_edited["Generator Capacity (MW)"].iloc[22] = 99

In [217]:
sa_edited["Technology"].iloc[28] = "Solar (Utility)"
sa_edited["Generator Capacity (MW)"].iloc[28] = 105

In [218]:
sa_edited["Technology"].iloc[33] = "Wind"
sa_edited["Generator Capacity (MW)"].iloc[33] = 210

### Write

In [220]:
sa_edited.to_csv(
    path+"edited/facilities_solar_wind_SA_edited.csv",
    index=False
)

# VIC

In [221]:
vic = pd.read_csv(
    path+"facilities_solar_wind_VIC.csv"
)

### Coordinates

In [222]:
vic.loc[
    (vic['Latitude'].isnull()) |
    (vic['Longitude'].isnull())
]

,Facility Name,Status,Region,Technology,Generator Capacity (MW),Latitude,Longitude
7,Cherry Tree,operating,Victoria,Wind,57.0,NaN,NaN
13,Elaine,operating,Victoria,Wind,83.0,NaN,NaN


In [223]:
vic_edited = vic.copy()

Cherry Tree

In [224]:
vic_edited["Latitude"].iloc[7] = -37.09961608519872
vic_edited["Longitude"].iloc[7] = 145.26938232111786

Elaine

In [225]:
vic_edited["Latitude"].iloc[13] = -37.732525496007005
vic_edited["Longitude"].iloc[13] = 144.0230869982597

### Wind and solar

In [226]:
vic_edited.loc[
    (vic_edited["Technology"] != "Solar (Utility)") &
    (vic_edited["Technology"] != "Wind")
]

,Facility Name,Status,Region,Technology,Generator Capacity (MW),Latitude,Longitude
4,Bulgana Green Power Hub,operating,Victoria,"Battery (Discharging),Wind",224.0,-37.039507,142.982799


In [227]:
vic_edited["Technology"].iloc[4] = "Wind"
vic_edited["Generator Capacity (MW)"].iloc[4] = 204

### Write

In [229]:
vic_edited.to_csv(
    path+"edited/facilities_solar_wind_VIC_edited.csv",
    index=False
)

# TAS

In [230]:
tas = pd.read_csv(
    path+"facilities_solar_wind_TAS.csv"
)

In [231]:
tas

,Facility Name,Status,Region,Technology,Generator Capacity (MW),Latitude,Longitude
0,Cattle Hill,operating,Tasmania,Wind,148,-42.154845,146.694693
1,Granville Harbour,operating,Tasmania,Wind,111,-41.802000,145.056000
2,Musselroe,operating,Tasmania,Wind,168,-40.777105,148.006315
3,Woolnorth,operating,Tasmania,Wind,140,-40.736375,144.694821


In [234]:
tas.to_csv(
    path+"edited/facilities_solar_wind_TAS_edited.csv",
    index=False
)

# WA

In [232]:
wa = pd.read_csv(
    path+"facilities_solar_wind_WEM.csv"
)

In [235]:
wa.to_csv(
    path+"edited/facilities_solar_wind_WEM_edited.csv",
    index=False
)